In [2]:
import geopandas as gpd
import json
import os
import requests
import urllib

from datetime import datetime
from pathlib import Path
from typing import Tuple

In [3]:
def shapefile_to_bbox(shapefile_path: Path, target_epsg: int = 4326) -> Tuple[float, float, float, float]:
    """
    Read a shapefile, reproject to the target CRS, and return its bounding box.

    Returns:
        (minx, miny, maxx, maxy) in the target CRS.
    """
    gdf = gpd.read_file(shapefile_path)
    gdf = gdf.to_crs(epsg=target_epsg)
    minx, miny, maxx, maxy = gdf.total_bounds
    return minx, miny, maxx, maxy

# path where annotations are located
ANNOTATIONS_PATH = Path("../landslides-detection-main/inventories/")
LOMBOK_PATH = ANNOTATIONS_PATH / "Lombok2018"
PHILIPPINES_PATH = ANNOTATIONS_PATH / "Philippines2019"
EMILIA_PATH = ANNOTATIONS_PATH / "EmiliaRomagna2023"
MICHOACAN_PATH = ANNOTATIONS_PATH / "Michoacan2022"

# inventories list
INVENTORIES = [
    {
        "name": "Lombok2018",
        "area": LOMBOK_PATH / "area.shp",
        "dates": [datetime(2018, 9, 9), datetime(2018, 9, 13), datetime(2018, 9, 27)],
    },
    {
        "name": "Philippines2019",
        "area": PHILIPPINES_PATH / "area_study.shp",
        "dates": [datetime(2020, 5, 2)],
    },
    {
        "name": "Michoacan2022",
        "area": EMILIA_PATH / "Area_rev.shp",
        "dates": [datetime(2022, 9, 25), datetime(2022, 10, 31)],
    },
    {
        "name": "EmiliaRomagna2023",
        "area": MICHOACAN_PATH / "investigated_area_LS_Michoacan2022.shp",
        "dates": [datetime(2023, 5, 22)],
    },
]


In [4]:
with open("./key.txt", "r") as f:
    API_key = f.read()

## Testing

In [5]:
API_URL = "https://api.planet.com/basemaps/v1/mosaics"
session = requests.Session()
session.auth = (API_key, "")

# get list of available mosaics
response = session.get(API_URL)
data = response.json()

In [6]:
BASE_URL = "https://api.planet.com/basemaps/v1/mosaics?api_key={}"
res = requests.get(BASE_URL.format(API_key))

for i, _ in enumerate(res.json()):
    print(res.json()["mosaics"][i]["name"])

global_monthly_2016_01_mosaic
global_monthly_2016_02_mosaic


In [7]:
y = "2016"
m = "03"

params = {
    "name__is" :f"global_monthly_{y}_{m}_mosaic"
}

res = session.get(API_URL, params = params)
print(res.status_code)

mosaic = res.json()
#print(json.dumps(mosaic, indent=2))

200


In [8]:
#get id
mosaic_id = mosaic['mosaics'][0]['id']
print(mosaic_id)
#get bbox for entire mosaic
mosaic_bbox = mosaic['mosaics'][0]['bbox']
#converting bbox to string for search params
string_bbox = ','.join(map(str, mosaic_bbox))

print('Mosaic id: '+ mosaic_id)
print('Mosaic bbox: '+ string_bbox)

cfd6a60f-ec1e-491d-a885-e06d43990bd5
Mosaic id: cfd6a60f-ec1e-491d-a885-e06d43990bd5
Mosaic bbox: -180,-55,180,76


## Solution - bbox coordinates are not set correctly

With valid coordinates, you will receive a list of items.
The download process then iterates through each item, retrieves its link, and downloads the corresponding content.

To select the month and year, change the "name__is" parameter.

In [11]:
y = "2018"
m = "09"

# Get bounding box as tuple and convert to string
area_bbox = shapefile_to_bbox(INVENTORIES[0]["area"])  # Returns (minx, miny, maxx, maxy)
area_bbox_str = ",".join(map(str, area_bbox))

print("BBox:", area_bbox_str)

# Prepare API request parameters
search_parameters = {
    "name__is": f"global_monthly_{y}_{m}_mosaic",
    "bbox": area_bbox_str,
    "level": str(18),
}

# Send request
res = session.get(API_URL, params=search_parameters, stream=True)
mosaic_id = mosaic['mosaics'][0]['id']

quads_url = f"{API_URL}/{mosaic_id}/quads"

# Handle response
print("Status code:", res.status_code)
try:
    data = res.json()
    #print("Response:", json.dumps(data, indent=2))

    if "items" in data:
        items = data["items"]
        #print("First quad item:\n", json.dumps(items[0], indent=2))
    else:
        print("No quads found for the given bbox.")
except Exception as e:
    print("Error parsing JSON:", e)

BBox: 116.03246503532793,-8.56332060429984,116.70731520051434,-8.257044874009
Status code: 200
No quads found for the given bbox.


In [12]:
# download
# a folder named "quads" has to be created beforehand
for i in items:
    link = i['_links']['download']
    name = i['id']
    name = name + '.tiff'
    DIR = 'quads/test'
    filename = os.path.join(DIR, name)

    # remove the comment to actually download images
    if not os.path.isfile(filename):
        pass
        #urllib.request.urlretrieve(link, filename)

NameError: name 'items' is not defined

# Automated

In [13]:
def get_previous_month(month_str, year_str):
    month = int(month_str)
    year = int(year_str)

    if month == 1:
        prev_month = 12
        prev_year = year - 1
    else:
        prev_month = month - 1
        prev_year = year

    return f"{prev_month:02}", str(prev_year)


def get_next_month(month_str, year_str):
    month = int(month_str)
    year = int(year_str)

    if month == 12:
        next_month = 1
        next_year = year + 1
    else:
        next_month = month + 1
        next_year = year

    return f"{next_month:02}", str(next_year)

In [16]:
# working
for el in INVENTORIES:
    dates = el["dates"]

    if len(dates) == 1:
        year = dates[0].year
        month = dates[0].month
        month_before, year_before = get_previous_month(month, year)
        month_after, year_after = get_next_month(month, year)
    else:
        first_year = dates[0].year
        last_year = dates[-1].year
        first_month = dates[0].month
        last_month = dates[-1].month
        month_before, year_before = get_previous_month(first_month, first_year)
        month_after, year_after = get_next_month(last_month, last_year)

    area_bbox = shapefile_to_bbox(el["area"])
    area_bbox_str = ",".join(map(str, area_bbox))

    for tag, y, m in [("before", year_before, month_before), ("after", year_after, month_after)]:
        mosaic_name = f"global_monthly_{y}_{m}_mosaic"

        print(f"Looking for mosaic: {mosaic_name}")
        
        # Get the mosaic ID
        res = session.get(API_URL, params={"name__is": mosaic_name}, stream=True)
        if res.status_code != 200:
            print(f"Failed to find mosaic '{mosaic_name}':", res.status_code)
            continue
        
        try:
            data = res.json()
            if "mosaics" not in data or not data["mosaics"]:
                print(f"No mosaics found for '{mosaic_name}'")
                continue
            mosaic_id = data["mosaics"][0]["id"]
        except Exception as e:
            print("Failed to parse mosaic response:", e)
            continue

        print(f"Using mosaic_id: {mosaic_id}")

        # Now get the quads for the bounding box
        quads_url = f"{API_URL}/{mosaic_id}/quads"
        res = session.get(quads_url, params={"bbox": area_bbox_str, "minimal": True}, stream=True)
        print(f"Status code for quads ({tag}):", res.status_code)
        
        try:
            data = res.json()
            items = data.get("items", [])
            if not items:
                print(f"No quads found for {tag}")
                continue
        except Exception as e:
            print("Error parsing quads JSON:", e)
            continue

        # Download quads
        for i in items:
            link = i['_links']['download']
            name = i['id'] + f"_{tag}.tiff"
            DIR = os.path.join('quads', el["name"])
            os.makedirs(DIR, exist_ok=True)
            filename = os.path.join(DIR, name)

            if not os.path.isfile(filename):
                print(f"Downloading {filename}")
                urllib.request.urlretrieve(link, filename)


Looking for mosaic: global_monthly_2018_08_mosaic
Using mosaic_id: 513bc1c8-25b3-4dee-b80c-cdc8e4b2099a
Status code for quads (before): 200
Looking for mosaic: global_monthly_2018_10_mosaic
Using mosaic_id: 28be1b87-3bda-4644-b087-e39c141e2634
Status code for quads (after): 200
Looking for mosaic: global_monthly_2020_04_mosaic
Using mosaic_id: 8b9dd650-dc7e-4082-a37f-867250a1cb6e
Status code for quads (before): 200
Looking for mosaic: global_monthly_2020_06_mosaic
Using mosaic_id: dfd557ec-a498-418a-9b1b-f7fa61f758b5
Status code for quads (after): 200


HTTPError: HTTP Error 504: Gateway Timeout

In [34]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os

def mergeTiff(path_root="./quads", suffix="before"):
    for name in os.listdir(path_root):
        path = os.path.join(path_root, name)
        if not os.path.isdir(path):
            continue
        
        # find every file which has _before.tiff at the end
        raster_files = glob.glob(f"{path_root}/{name}/*_{suffix}.tiff")
        print("reading:", f"{path_root}/{name}/*_{suffix}.tiff")
        
        #print("Found raster files:", raster_files)
        
        src_files_to_mosaic = [rasterio.open(fp) for fp in raster_files]
        
        # merge
        mosaic, out_transform = merge(src_files_to_mosaic)
        
        # Step 4: Copy metadata and update it
        out_meta = src_files_to_mosaic[0].meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_transform
        })
        
        # Step 5: Write to a new file
        with rasterio.open(f"./quads/{name}/{name.lower()}_{suffix}_merged.tiff", "w", **out_meta) as dest:
            print(f"{name.lower()}_{suffix}_merged.tiff mosaic written in {path}")
            dest.write(mosaic)

mergeTiff()
mergeTiff(suffix="after")

reading: ./quads/Philippines2019/*_before.tiff
philippines2019_before_merged.tiff mosaic written in ./quads/Philippines2019
reading: ./quads/EmiliaRomagna2023/*_before.tiff
emiliaromagna2023_before_merged.tiff mosaic written in ./quads/EmiliaRomagna2023
reading: ./quads/Lombok2018/*_before.tiff
lombok2018_before_merged.tiff mosaic written in ./quads/Lombok2018
reading: ./quads/Michoacan2022/*_before.tiff
michoacan2022_before_merged.tiff mosaic written in ./quads/Michoacan2022
reading: ./quads/Philippines2019/*_after.tiff
philippines2019_after_merged.tiff mosaic written in ./quads/Philippines2019
reading: ./quads/EmiliaRomagna2023/*_after.tiff
emiliaromagna2023_after_merged.tiff mosaic written in ./quads/EmiliaRomagna2023
reading: ./quads/Lombok2018/*_after.tiff
lombok2018_after_merged.tiff mosaic written in ./quads/Lombok2018
reading: ./quads/Michoacan2022/*_after.tiff
michoacan2022_after_merged.tiff mosaic written in ./quads/Michoacan2022
